<a href="https://colab.research.google.com/github/znumunz/BSC_DPDM2025/blob/main/Ch3_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**import library**

In [ ]:
import pandas as pd

#**Import data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
flood = pd.read_csv('/content/drive/MyDrive/BSC_DPDM2025/data/flooding database.csv')
flood.head()

# **Data Cleansing**

##**Missing Data**

### check missing data

In [ ]:
flood.isnull().sum()

In [ ]:
flood.shape

### Drop Record ที่มี Missing

In [ ]:
flood1 = flood.dropna()
flood1

In [ ]:
flood1.shape

In [ ]:
flood.columns

###Drop เฉพาะ การเกิด missing ในตัวแปรที่เราจะใช้

In [ ]:
flood2 = flood.dropna(subset=['dfo_country', 'dfo_other_country','dfo_began','dfo_ended','dfo_dead','dfo_cause','dfo_severity'])
flood2.head()


In [ ]:
flood2.shape

### เติม dummy

In [ ]:
flood[flood.isnull()['dfo_other_country']]

In [ ]:
flood_fill_dummy = flood.fillna({'dfo_other_country':'Unknow'})
flood_fill_dummy

In [ ]:
flood_fill_dummy.isnull().sum()

### เติม Missing ด้วย ค่ากลาง

In [ ]:
flood['dfo_cause'].value_counts()

In [ ]:
flood_fillna_mode = flood_fill_dummy.fillna({'dfo_cause':'Heavy rain'})
flood_fillna_mode

In [ ]:
flood_fillna_mode.isnull().sum()

### เติม Dummy ด้วยค่ากลางของแต่ละกลุ่ม

####
dfo_severity = 1.0 เติม dfo_cause 'Havy rain'

dfo_severity = 1.5 เติม dfo_cause
'Havy rain'

dfo_severity = 2.0 เติม dfo_cause 'Tropical Storm, Surge'

In [ ]:
flood.groupby(['dfo_severity','dfo_cause']).count()

In [ ]:
flood_sever1 = flood[flood['dfo_severity']==1.0]
flood_sever15 = flood[flood['dfo_severity']==1.5]
flood_sever2 = flood[flood['dfo_severity']==2.0]

In [ ]:
flood_sever1 = flood_sever1.fillna({'dfo_cause':'Heavy rain'})
flood_sever15 = flood_sever15.fillna({'dfo_cause':'Heavy rain'})
flood_sever2 = flood_sever2.fillna({'dfo_cause':'Tropical Storm, Surge'})

In [ ]:
flood_fillgroup = pd.concat([flood_sever1,flood_sever15,flood_sever2])
flood_fillgroup.head()

In [ ]:
flood_fillgroup.isnull().sum()

# Noisy Data

In [ ]:
flood.head()

In [ ]:
plt.plot(flood['gfd_area'])

## ใช้ hostogram ดู Bin ที่เหมาะสม

In [ ]:
outputhist = plt.hist(flood['gfd_area'], 30);

In [ ]:
outputhist[1]

## binning

In [ ]:
def get_bin_for_value(value, bin_edges):
    for i in range(len(bin_edges) - 1):
        if bin_edges[i] <= value < bin_edges[i+1]:
            # Return the midpoint immediately when the bin is found
            return bin_edges[i] + ((bin_edges[i+1] - bin_edges[i]) / 2)
    # Handle the case where the value might be exactly the upper bound of the last bin
    if value == bin_edges[-1]:
        return bin_edges[-2] + ((bin_edges[-1] - bin_edges[-2]) / 2)
    # If the value is outside all defined bins, return None
    return None

In [ ]:
get_bin_for_value(2000,outputhist[1])

In [ ]:
# Ensure 'bins' is available from outputhist[1] if not already in global scope
if 'outputhist' not in locals():
    # This part should ideally not be needed if previous cells were run
    # but included for robustness in case of partial execution.
    import matplotlib.pyplot as plt
    outputhist = plt.hist(flood['gfd_area'], 30)
bins = outputhist[1]

# Apply the get_bin_for_value function to the 'gfd_area' column
flood['gfd_area_binned_midpoint'] = flood['gfd_area'].apply(lambda x: get_bin_for_value(x, bins))

# Display the DataFrame with the new column
display(flood[['gfd_area', 'gfd_area_binned_midpoint']].head())

In [ ]:
# Ensure 'bins' is available from outputhist[1] if not already in global scope
# if 'outputhist' not in locals():
#     import matplotlib.pyplot as plt
#     outputhist = plt.hist(flood['gfd_area'], 30)
bins = outputhist[1]

# Apply the get_bin_for_value function to the 'gfd_area' column
flood['gfd_area_binned_midpoint'] = flood['gfd_area'].apply(lambda x: get_bin_for_value(x, bins))

# Display the DataFrame with the new column
display(flood[['gfd_area', 'gfd_area_binned_midpoint']].head())

In [ ]:
flood['gfd_area'].sort_values()

In [ ]:
plt.figure(figsize=(10, 10)) # Adjust width (first number) and height (second number) as desired
plt.subplot(2,1,1)
plt.plot(flood['gfd_area'].sort_values().reset_index(drop=True))
plt.title('Original gfd_area')
plt.subplot(2,1,2)
plt.plot(flood['gfd_area_binned_midpoint'].sort_values().reset_index(drop=True))
plt.title('Binned gfd_area Midpoints')
plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
plt.show()

## Oulier Remval

In [ ]:
plt.boxplot(flood['gfd_area'], vert=False);

In [ ]:
outputbox = plt.boxplot(flood['gfd_area'], vert=False,whis=10)

In [ ]:
outputbox['whiskers'][1].get_xdata()

In [ ]:
outputbox['whiskers'][0].get_xdata()

In [ ]:
outputbox['fliers'][0].get_xdata()

## remove outliers

In [ ]:
flood_nooutlier = flood[flood['gfd_area']<outputbox['whiskers'][1].get_xdata()[1]]
flood_nooutlier.head()

In [ ]:
plt.boxplot(flood_nooutlier['gfd_area'], vert=False,whis = 10);

### ตามหาค่า outlier

In [ ]:
flood[flood['gfd_area']==outputbox['fliers'][0].get_xdata()[0]]